### Parse CRR3

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import logging
import tiktoken
import numpy as np
from openai import OpenAI
import os
from tqdm import tqdm
import time

from utils import print_summary, check_article, DocumentCategorizer, CostTracker
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from sklearn.metrics.pairwise import cosine_similarity
import yaml

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)

# Load the configuration
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [2]:
class EURLexParser:
    """Simplified EUR-Lex document parser that correctly handles all numbering"""
    
    # Define hierarchy levels in order
    HIERARCHY_LEVELS = ['part', 'title', 'chapter', 'section', 'subsection']
    
    def __init__(self):
        # Initialize hierarchy with both number and heading for each level
        self.hierarchy = {f'{level}{"_heading" if is_heading else ""}': '' 
                         for level in self.HIERARCHY_LEVELS 
                         for is_heading in [False, True]}
        self.articles = []
        self.pending_division = None
        
        # Initialize tiktoken encoder if available
        self.encoding = tiktoken.get_encoding("cl100k_base")
    
    def clean_text(self, text):
        """Clean text by removing modification markers and extra whitespace"""
        if not text:
            return ""
        
        # Remove modification markers and normalize whitespace
        text = re.sub(r'►[A-Z]\d+|◄|▼[A-Z]\d+|▲', '', text)
        text = text.replace('\xa0', ' ')
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'(?<!\d)\s+\.', '.', text)
        
        return text.strip()
    
    def reset_hierarchy_below(self, level):
        """Reset all hierarchy levels below the given level"""
        levels = self.HIERARCHY_LEVELS
        start_idx = levels.index(level) + 1 if level in levels else 0
        
        for lvl in levels[start_idx:]:
            self.hierarchy[lvl] = ''
            self.hierarchy[f'{lvl}_heading'] = ''
    
    def update_hierarchy(self, text1, text2=None):
        """Update hierarchy based on division text"""
        text1_upper = text1.upper()
        
        # Determine the hierarchy level
        for level in self.HIERARCHY_LEVELS:
            if level.upper() in text1_upper and (
                text1.strip().upper().startswith(level.upper()) or 
                f' {level.upper()} ' in f' {text1_upper} '
            ):
                # Special handling for PART with inline heading
                if level == 'part' and not text2:
                    match = re.match(r'^(PART\s+[A-Z]+)\s*[-\u2013\u2014]\s*(.+)$', text1, re.IGNORECASE)
                    if match:
                        self.hierarchy[level] = match.group(1).strip()
                        self.hierarchy[f'{level}_heading'] = match.group(2).strip()
                    else:
                        self.hierarchy[level] = text1
                        self.hierarchy[f'{level}_heading'] = ''
                else:
                    self.hierarchy[level] = text1
                    self.hierarchy[f'{level}_heading'] = text2 or ''
                
                self.reset_hierarchy_below(level)
                logging.info(f"Found {level.capitalize()}: {text1} - {text2 or ''}")
                return True
        return False
    
    def extract_grid_list(self, grid_container):
        """Extract content from grid-container preserving all numbering"""
        items = []
        
        # Process pairs of grid columns
        column1_divs = grid_container.find_all('div', 
            class_=lambda c: c and 'grid-list-column-1' in c, recursive=False)
        
        for col1_div in column1_divs:
            marker = col1_div.get_text(strip=True)
            
            # Find corresponding column-2
            col2_div = col1_div.find_next_sibling(
                'div', class_=lambda c: c and 'grid-list-column-2' in c)
            
            if col2_div:
                content_parts = []
                
                # Process all children of column 2
                for child in col2_div.children:
                    if hasattr(child, 'name'):
                        if child.name == 'div' and 'grid-container' in ' '.join(child.get('class', [])):
                            # Nested grid list
                            nested_content = self.extract_grid_list(child)
                            if nested_content:
                                content_parts.append(nested_content)
                        else:
                            # Other elements
                            text = self.clean_text(child.get_text())
                            if text:
                                content_parts.append(text)
                    else:
                        # Text node
                        text = str(child).strip()
                        if text:
                            content_parts.append(text)
                
                content = ' '.join(content_parts)
                if marker and content:
                    items.append(f"{marker} {content}")
        
        return ' '.join(items)
    
    def extract_article_content(self, article_div):
        """Extract complete article content preserving all numbering"""
        # Get article title
        title_elem = article_div.find('p', class_='title-article-norm')
        if not title_elem:
            return None
        
        article_heading = self.clean_text(title_elem.get_text())
        
        # Extract article number
        match = re.match(r'Article\s+(\d+[a-zA-Z]*)', article_heading, re.IGNORECASE)
        if not match:
            return None
        article_number = match.group(1)
        
        # Get subtitle if exists
        subtitle_elem = article_div.find('p', class_='stitle-article-norm')
        if subtitle_elem:
            subtitle = self.clean_text(subtitle_elem.get_text())
            if subtitle:
                article_heading += f" - {subtitle}"
        
        # Extract all content parts
        content_parts = []
        
        # Process all direct children that are norm elements
        for child in article_div.children:
            if not hasattr(child, 'name'):
                continue
            
            # Skip elements we don't want
            if (child.name == 'p' and 
                any(cls in child.get('class', []) for cls in 
                    ['modref', 'title-article-norm', 'stitle-article-norm'])):
                continue
            
            # Process norm paragraphs
            if child.name == 'p' and 'norm' in child.get('class', []):
                text = self.clean_text(child.get_text())
                if text:
                    content_parts.append(text)
            
            # Process grid containers
            elif child.name == 'div' and 'grid-container' in ' '.join(child.get('class', [])):
                grid_content = self.extract_grid_list(child)
                if grid_content:
                    content_parts.append(grid_content)
            
            # Process div.norm elements
            elif child.name == 'div' and 'norm' in child.get('class', []):
                # Get paragraph number if exists
                para_span = child.find('span', class_='no-parag')
                para_num = para_span.get_text(strip=True) if para_span else ""
                
                # Extract content
                text_parts = []
                for elem in child.children:
                    if hasattr(elem, 'name'):
                        if elem.name == 'span' and 'no-parag' in elem.get('class', []):
                            continue
                        elif elem.name == 'div' and 'grid-container' in ' '.join(elem.get('class', [])):
                            grid_content = self.extract_grid_list(elem)
                            if grid_content:
                                text_parts.append(grid_content)
                        else:
                            text = self.clean_text(elem.get_text())
                            if text:
                                text_parts.append(text)
                    else:
                        text = str(elem).strip()
                        if text:
                            text_parts.append(text)
                
                combined_text = ' '.join(text_parts)
                if combined_text:
                    content_parts.append(f"{para_num} {combined_text}" if para_num else combined_text)
        
        return {
            'article_number': article_number,
            'article_heading': article_heading,
            'text': ' '.join(content_parts),
            **self.hierarchy  # Add current hierarchy
        }
    
    def parse_document(self, url):
        """Parse EUR-Lex document from URL"""
        logging.info(f"Fetching document from: {url}")
        
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all relevant elements
        all_elements = soup.find_all(['div', 'p'], class_=True)
        
        for element in all_elements:
            classes = ' '.join(element.get('class', []))
            text = self.clean_text(element.get_text())
            
            if not text:
                continue
            
            # Handle title-division pairs
            if 'title-division-1' in classes:
                self.pending_division = text
            elif 'title-division-2' in classes and self.pending_division:
                self.update_hierarchy(self.pending_division, text)
                self.pending_division = None
            
            # Handle other section types
            elif any(cls in classes for cls in ['ti-section-1', 'sti-section-1']):
                self.update_hierarchy(text)
            
            # Process articles
            elif 'eli-subdivision' in classes and element.name == 'div':
                article_data = self.extract_article_content(element)
                if article_data:
                    self.articles.append(article_data)
                    logging.info(f"Processed Article {article_data['article_number']}")
        
        return self.create_dataframe()
    
    def create_dataframe(self):
        """Create and enhance the final DataFrame"""
        if not self.articles:
            return pd.DataFrame()
        
        df = pd.DataFrame(self.articles)
        
        # Rename columns for consistency
        df.rename(columns={
            'article_number': 'Article_Number',
            'article_heading': 'Article_Heading',
            'text': 'Text'
        }, inplace=True)
        
        # Add validation columns
        if self.encoding:
            df['Token_Count'] = df['Text'].apply(lambda x: len(self.encoding.encode(x)) if x else 0)
        else:
            df['Token_Count'] = df['Text'].apply(lambda x: len(x) // 4 if x else 0)
        
        df['Ends_With_Dot'] = df['Text'].apply(lambda x: x.strip().endswith('.') if x else False)
        df['Text_With_Pagebreaks'] = df['Text'].apply(self.add_pagebreaks_to_text)
        
        # Capitalize hierarchy column names
        for col in df.columns:
            if col not in ['Article_Number', 'Article_Heading', 'Text', 'Token_Count', 'Ends_With_Dot', 'Text_With_Pagebreaks']:
                new_name = col.replace('_', ' ').title().replace(' ', '_')
                df.rename(columns={col: new_name}, inplace=True)
        
        # Remove empty hierarchy columns
        hierarchy_cols = [col for col in df.columns if col not in 
                         ['Article_Number', 'Article_Heading', 'Text', 'Token_Count', 'Ends_With_Dot', 'Text_With_Pagebreaks']]
        
        for col in hierarchy_cols:
            if df[col].str.strip().eq('').all():
                df.drop(columns=[col], inplace=True)
        
        # Reorder columns: hierarchy → validation → article info → text with pagebreaks
        hierarchy_cols = [col for col in df.columns if col not in 
                         ['Article_Number', 'Article_Heading', 'Text', 'Token_Count', 'Ends_With_Dot', 'Text_With_Pagebreaks']]
        
        hierarchy_number_cols = [col for col in hierarchy_cols if not col.endswith('_Heading')]
        hierarchy_heading_cols = [col for col in hierarchy_cols if col.endswith('_Heading')]
        
        new_order = (hierarchy_number_cols + hierarchy_heading_cols + 
                    ['Token_Count', 'Ends_With_Dot', 'Article_Number', 'Article_Heading', 'Text', 'Text_With_Pagebreaks'])
        
        return df[new_order]
    
    def add_pagebreaks_to_text(self, text):
        """Add pagebreaks before subheadings"""
        if not text:
            return text
        
        # Apply patterns for different numbering types
        patterns = [
            (r'(?<=[.;,\s])(\s*)(\([ivxlcdm]+\))', r'\n\n\2'),  # (i), (ii), etc.
            (r'(?<=[.;,\s])(\s*)(\([a-z]\))', r'\n\n\2'),       # (a), (b), etc.
            (r'(?<=[.;,\s])(\s*)(\(\d+\))', r'\n\n\2'),         # (1), (2), etc.
        ]
        
        for pattern, replacement in patterns:
            text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
        
        return re.sub(r'\n{3,}', '\n\n', text).strip()


def parse_eurlex_document(url, output_file='eurlex_parsed.csv', save_csv=True):
    """Parse EUR-Lex document from URL"""
    parser = EURLexParser()
    
    try:
        df = parser.parse_document(url)
        df['id'] = df.index
        
        print(f"\nParsing completed! Total articles found: {len(df)}")
            
        # Print summary
        print_summary(df)
            
        return df
            
    except Exception as e:
        print(f"Error parsing document: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()

In [3]:
# Usage example:
url = "https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:02013R0575-20250101"
df = parse_eurlex_document(url)

print("\nCategorizing documents...")
cost_tracker = CostTracker(config)
llm = ChatOpenAI(model=config["llm_model"], temperature=0)

categorizer = DocumentCategorizer(config["Category_1"], config["Category_2"], llm, cost_tracker)
df = categorizer.categorize_dataset(df)

# Quick checks
if not df.empty:
    print(f"\n✅ Successfully parsed {len(df)} articles")
    check_article(df, '5a')

2025-06-15 22:18:15,567 - INFO - Fetching document from: https://eur-lex.europa.eu/legal-content/EN/TXT/HTML/?uri=CELEX:02013R0575-20250101
2025-06-15 22:18:21,458 - INFO - Processed Article 1
2025-06-15 22:18:21,460 - INFO - Found Part: PART ONE - GENERAL PROVISIONS
2025-06-15 22:18:21,460 - INFO - Found Title: TITLE I - SUBJECT MATTER, SCOPE AND DEFINITIONS
2025-06-15 22:18:21,462 - INFO - Processed Article 1
2025-06-15 22:18:21,464 - INFO - Processed Article 2
2025-06-15 22:18:21,465 - INFO - Processed Article 3
2025-06-15 22:18:21,480 - INFO - Processed Article 4
2025-06-15 22:18:21,527 - INFO - Processed Article 5
2025-06-15 22:18:21,540 - INFO - Processed Article 5a
2025-06-15 22:18:21,546 - INFO - Found Title: TITLE II - LEVEL OF APPLICATION OF REQUIREMENTS
2025-06-15 22:18:21,547 - INFO - Found Chapter: CHAPTER 1 - Application of requirements on an individual basis
2025-06-15 22:18:21,552 - INFO - Processed Article 6
2025-06-15 22:18:21,559 - INFO - Processed Article 7
2025-06-


Parsing completed! Total articles found: 743

DOCUMENT STRUCTURE SUMMARY

Document hierarchy:
  - Part: 12 unique (with headings)
  - Title: 8 unique (with headings)
  - Chapter: 11 unique (with headings)
  - Section: 10 unique (with headings)
  - Subsection: 6 unique (with headings)

Validation summary:
  - Total tokens: 371,237
  - Average tokens per article: 500
  - Articles ending with period: 712/743

First 5 articles:
  Article_Number                                    Article_Heading  \
0              1                                  Article 1 - Scope   
1              1                                  Article 1 - Scope   
2              2                     Article 2 - Supervisory powers   
3              3  Article 3 - Application of stricter requiremen...   
4              4                            Article 4 - Definitions   

   Token_Count  
0            0  
1          215  
2          322  
3           30  
4        12717  

Numbering patterns found:
  - Articles wi

Categorizing Documents: 100%|████████████████████████████████████████████████████████| 743/743 [10:04<00:00,  1.23it/s]


✅ Successfully parsed 743 articles

Article 5a: Article 5a - Definitions specific to crypto-assets
Tokens: 683, Ends with dot: True

Numbering found:
  Main (1), (2): (1), (1), (5), (2), (1) ...
  Letters (a), (b): (a), (b), (c), (d), (e) ...
  Roman (i), (ii): (c), (d), (i)

Text preview:
------------------------------------------------------------
For the purposes of this Regulation, the following definitions apply: (1) ‘crypto-asset’ means a crypto-asset as defined in Article 3(1), point (5), of Regulation (EU) 2023/1114 of the European Parliament and of the Council ( 20 ) that is not a central bank digital currency; (2) ‘electronic money token’ or ‘e-money token’ means an electronic money token or e-money token as defined in Article 3(1), point (7), of Regulation (EU) 2023/1114; (3) ‘crypto-asset exposure’ means an asset or an off-balan...


In [9]:
df.columns

Index(['Part', 'Title', 'Chapter', 'Section', 'Subsection', 'Part_Heading',
       'Title_Heading', 'Chapter_Heading', 'Section_Heading',
       'Subsection_Heading', 'Token_Count', 'Ends_With_Dot', 'Article_Number',
       'Article_Heading', 'Text', 'Text_With_Pagebreaks', 'id', 'Category_1',
       'Category_2', 'combined_text', 'embedding'],
      dtype='object')

In [4]:
# Initialize OpenAI client
client = OpenAI()

# Enable tqdm for pandas
tqdm.pandas()

# Create combined text column with column names as prefixes
columns_to_combine = ['Part_Heading', 'Title_Heading', 'Chapter_Heading', 
                      'Section_Heading', 'Subsection_Heading', 'Article_Heading', 
                      'Text_With_Pagebreaks']

def combine_with_labels(row):
    parts = []
    for col in columns_to_combine:
        if pd.notna(row[col]) and row[col] != '':
            parts.append(f"{col}: {row[col]}")
    return '\n'.join(parts)

print("Combining text columns...")
df['combined_text'] = df.progress_apply(combine_with_labels, axis=1)

# Truncate text to 8000 characters (max input size is 8192 tokens)
df['combined_text'] = df['combined_text'].str[:3000]

# Function to get embedding with retry logic
def get_embeddings(text, max_retries=3):
    for attempt in range(max_retries):
        try:
            return client.embeddings.create(input=text, model="text-embedding-3-small").data[0].embedding
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(1)  # Wait 1 second before retry
                continue
            else:
                raise e

# Generate embeddings with progress bar
print("Generating embeddings...")
df['embedding'] = df['combined_text'].progress_apply(get_embeddings)

# Create data folder and save
os.makedirs('data', exist_ok=True)
df.to_parquet('data/df_with_embeddings.parquet')

print("Done! Saved to data/df_with_embeddings.parquet")

Combining text columns...


100%|█████████████████████████████████████████████████████████████████████████████| 743/743 [00:00<00:00, 12419.22it/s]


Generating embeddings...


100%|████████████████████████████████████████████████████████████████████████████████| 743/743 [03:19<00:00,  3.73it/s]

Done! Saved to data/df_with_embeddings.parquet


In [5]:
pd.options.display.max_colwidth = 1000
df[5:].head(5)

,Part,Title,Chapter,Section,Subsection,Part_Heading,Title_Heading,Chapter_Heading,Section_Heading,Subsection_Heading,...,Ends_With_Dot,Article_Number,Article_Heading,Text,Text_With_Pagebreaks,id,Category_1,Category_2,combined_text,embedding
5,PART ONE,TITLE I,,,,GENERAL PROVISIONS,"SUBJECT MATTER, SCOPE AND DEFINITIONS",,,,...,True,5,Article 5 - Definitions specific to capital requirements for credit risk,"For the purposes of Part Three, Title II, the following definitions shall apply: (1) ‘exposure’ means an asset or off-balance sheet item; (2) ‘loss’ means economic loss, including material discount effects, and material direct and indirect costs associated with collecting on the instrument; (3) ‘expected loss’ or ‘EL’ means the ratio, related to a single facility, of the amount expected to be lost on an exposure from any of the following: (a) a potential default of an obligor over a one-year period to the amount outstanding at default; (b) a potential dilution event over a one-year period to the amount outstanding at the date of occurrence of the dilution event; (4) ‘credit obligation’ means any obligation arising from a credit contract, including principal, accrued interest and fees, owed by an obligor; (5) ‘credit exposure’ means any on- or off -balance-sheet item, that results, or may result, in a credit obligation; (6) ‘facility’ or ‘credit facility’ means a credit exposure ari...","For the purposes of Part Three, Title II, the following definitions shall apply: \n\n(1) ‘exposure’ means an asset or off-balance sheet item;\n\n(2) ‘loss’ means economic loss, including material discount effects, and material direct and indirect costs associated with collecting on the instrument;\n\n(3) ‘expected loss’ or ‘EL’ means the ratio, related to a single facility, of the amount expected to be lost on an exposure from any of the following: \n\n(a) a potential default of an obligor over a one-year period to the amount outstanding at default;\n\n(b) a potential dilution event over a one-year period to the amount outstanding at the date of occurrence of the dilution event;\n\n(4) ‘credit obligation’ means any obligation arising from a credit contract, including principal, accrued interest and fees, owed by an obligor;\n\n(5) ‘credit exposure’ means any on- or off -balance-sheet item, that results, or may result, in a credit obligation;\n\n(6) ‘facility’ or ‘credit facility’ m...",5,Credit Risk,Risk Weights and Correlations,"Part_Heading: GENERAL PROVISIONS\nTitle_Heading: SUBJECT MATTER, SCOPE AND DEFINITIONS\nArticle_Heading: Article 5 - Definitions specific to capital requirements for credit risk\nText_With_Pagebreaks: For the purposes of Part Three, Title II, the following definitions shall apply: \n\n(1) ‘exposure’ means an asset or off-balance sheet item;\n\n(2) ‘loss’ means economic loss, including material discount effects, and material direct and indirect costs associated with collecting on the instrument;\n\n(3) ‘expected loss’ or ‘EL’ means the ratio, related to a single facility, of the amount expected to be lost on an exposure from any of the following: \n\n(a) a potential default of an obligor over a one-year period to the amount outstanding at default;\n\n(b) a potential dilution event over a one-year period to the amount outstanding at the date of occurrence of the dilution event;\n\n(4) ‘credit obligation’ means any obligation arising from a credit contract, including principal, accrue...","[-0.006079161074012518, 0.03316662088036537, 0.06238935887813568, -0.001397286425344646, -0.010631849989295006, 0.020491553470492363, -0.0008857399807311594, 0.027820996940135956, 0.04758792370557785, -0.01318586990237236, 0.02632422186434269, -0.05939580872654915, -0.014231235720217228, -0.011689094826579094, -0.018555250018835068, 0.02413845621049404, 0.013732311315834522, 0.057732727378606796, 0.035803794860839844, 0.0333566851913929, 0.029650388285517693, 0.03910620138049126, -0.023995906114578247, 0.04202847555279732, -0.01824639

In [6]:
len(df)

743

In [7]:
pd.options.display.max_colwidth = 100000
df[df['Ends_With_Dot']==False]

Part      Title     Chapter    Section    Subsection  \
0                                                                 
59     PART TWO    TITLE I   CHAPTER 3  Section 2                 
142  PART THREE   TITLE II   CHAPTER 2  Section 2                 
159  PART THREE   TITLE II   CHAPTER 2  Section 2                 
164  PART THREE   TITLE II   CHAPTER 2  Section 3                 
184  PART THREE   TITLE II   CHAPTER 3  Section 3                 
193  PART THREE   TITLE II   CHAPTER 3  Section 5                 
210  PART THREE   TITLE II   CHAPTER 3  Section 6                 
246  PART THREE   TITLE II   CHAPTER 4  Section 4                 
251  PART THREE   TITLE II   CHAPTER 4  Section 4                 
272  PART THREE   TITLE II   CHAPTER 5  Section 3  Subsection 1   
280  PART THREE   TITLE II   CHAPTER 5  Section 3  Subsection 3   
282  PART THREE   TITLE II   CHAPTER 5  Section 3  Subsection 3   
284  PART THREE   TITLE II   CHAPTER 5  Section 3  Subsection 3   
298  PART THREE   TITLE II   CHAPTER 6  Section 1                 
308  PART THREE   TITLE II   CHAPTER 6  Section 3                 
312  PART THREE   TITLE II   CHAPTER 6  Section 3                 
313  PART THREE   TITLE II   CHAPTER 6  Section 3                 
315  PART THREE   TITLE II   CHAPTER 6  Section 3                 
347  PART THREE   TITLE II   CHAPTER 6  Section 9                 
352  PART THREE  TITLE III   CHAPTER 1                            
363  PART THREE  TITLE III   CHAPTER 2                            
399  PART THREE   TITLE IV  CHAPTER 1a  Section 6  Subsection 1   
400  PART THREE   TITLE IV  CHAPTER 1a  Section 6  Subsection 1   
404  PART THREE   TITLE IV  CHAPTER 1a  Section 6  Subsection 1   
407  PART THREE   TITLE IV  CHAPTER 1a  Section 6  Subsection 1   
409  PART THREE   TITLE IV  CHAPTER 1a  Section 6  Subsection 1   
485  PART THREE   TITLE VI                                        
493  PART THREE   TITLE VI                                        
513   PART FOUR                                                   
740    PART TEN  TITLE III                                        

                                                 Part_Heading  \
0                                                               
59                         OWN FUNDS AND ELIGIBLE LIABILITIES   
142                                      CAPITAL REQUIREMENTS   
159                                      CAPITAL REQUIREMENTS   
164                                      CAPITAL REQUIREMENTS   
184                                      CAPITAL REQUIREMENTS   
193                                      CAPITAL REQUIREMENTS   
210                                      CAPITAL REQUIREMENTS   
246                                      CAPITAL REQUIREMENTS   
251                                      CAPITAL REQUIREMENTS   
272                                      CAPITAL REQUIREMENTS   
280                                      CAPITAL REQUIREMENTS   
282                                      CAPITAL REQUIREMENTS   
284                                      CAPITAL REQUIREMENTS   
298                                      CAPITAL REQUIREMENTS   
308                                      CAPITAL REQUIREMENTS   
312                                      CAPITAL REQUIREMENTS   
313                                      CAPITAL REQUIREMENTS   
315                                      CAPITAL REQUIREMENTS   
347                                      CAPITAL REQUIREMENTS   
352                                      CAPITAL REQUIREMENTS   
363                                      CAPITAL REQUIREMENTS   
399                                      CAPITAL REQUIREMENTS   
400                                      CAPITAL REQUIREMENTS   
404                                      CAPITAL REQUIREMENTS   
407                                      CAPITAL REQUIREMENTS   
409                                      CAPITAL REQUIREMENTS   
485                                      CAPIT

In [8]:
df.to_excel('data/df_with_embeddings.xlsx', index=False)